In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import codecs, json

In [5]:
#importing all csv files
domarar = pd.read_csv('csv/blak-domarar.csv', sep=';', header=0)
einstaklingar = pd.read_csv('csv/blak-einstaklingar.csv', sep=';', header=0)
forsvarsmenn = pd.read_csv('csv/blak-forsvarsmenn.csv', sep=';', header=0)
lid = pd.read_csv('csv/blak-lid.csv', sep=';', header=0)
lidimoti = pd.read_csv('csv/blak-lidimoti.csv', sep=';', header=0)
lidsmenn = pd.read_csv('csv/blak-lidsmenn.csv', sep=';', header=0)
lidsstjorar = pd.read_csv('csv/blak-lidsstjorar.csv', sep=';', header=0)
thjalfarar = pd.read_csv('csv/blak-thjalfarar.csv', sep=';', header=0)
mot = pd.read_csv('csv/blak-mot.csv', sep=';', header=0)

In [6]:
# drop all SyndarLids with an ID (SyndarlidID)
# (the reason for not dropping using SyndarLid is because I don't trust that column to be inserted correctly with [0,1])
lid = lid[lid['SyndarlidID'].isna()]
# then dropping those two columns because we don't want virtual teams
lid = lid.drop(columns=['SyndarLid', 'SyndarlidID'])

In [7]:
## data
#print(help(einstaklingar))
#print(domarar)
#print(einstaklingar)
#print(forsvarsmenn)
#print(lid.columns)
#print(lidimoti)
#print(lidsmenn)
#print(lidsstjorar)
#print(mot)
#print(thjalfarar)

## describe
#print(domarar.describe())
#print(einstaklingar.describe())
#print(forsvarsmenn.describe())
#print(lid.describe())
#print(lidimoti.describe())
#print(lidsmenn.describe())
#print(lidsstjorar.describe())
#print(mot.describe())
#print(thjalfarar.describe())

## dtypes
#domarar.dtypes
#einstaklingar.dtypes
#forsvarsmenn.dtypes
#lid.dtypes
#lidimoti.dtypes
#lidsmenn.dtypes
#lidsstjorar.dtypes
#mot.dtypes
#thjalfarar.dtypes

In [8]:
# All duplicated birthdays
duplicated_einstaklingar = einstaklingar[einstaklingar.duplicated(subset=['Nafn', 'Fdagur', 'Kyn'], keep=False)]
duplicated_fdagur_kyn_einstaklingar = einstaklingar[einstaklingar.duplicated(subset=['Fdagur', 'Kyn'], keep=False)]
#print(duplicated_fdagur_kyn_einstaklingar)

In [9]:
duplicate_dict = defaultdict(dict)

for index, row in duplicated_fdagur_kyn_einstaklingar.iterrows():
    full_name = row['Nafn']
    first_name = full_name.split()[0]
    first_name_lowercase = first_name.lower()
    Fdagur_date = row['Fdagur'].split()[0]
    
    #dict_by_firstname = duplicate_dict[first_name_lowercase] 
    if first_name_lowercase in duplicate_dict.keys():
        if Fdagur_date in duplicate_dict[first_name_lowercase].keys():
            duplicate_dict[first_name_lowercase][Fdagur_date].append(row.values)
        else:
            duplicate_dict[first_name_lowercase][Fdagur_date] = [row.values]
    else:
        #if Fdagur does not exist ->
        duplicate_dict[first_name_lowercase][Fdagur_date] = [row.values]
        

In [10]:
#dict(duplicated_fdagur_kyn_einstaklingar)

# all duplicates
#duplicate_dict

# all duplicates for lúðvík
#duplicate_dict["lúðvík"]

{'1996-09-02': [array([4160, 'Lúðvík', '1996-09-02 00:00:00.000', 'kk ', nan, nan, nan,
         nan, nan, nan, nan, nan, '2013-04-28 10:50:35.960', 0.0],
        dtype=object),
  array([4167, 'Lúðvík Már Matthíasson', '1996-09-02 00:00:00.000', 'kk ',
         'HK', nan, nan, nan, nan, nan, nan, nan, '2013-09-03 21:00:29.950',
         0.0], dtype=object)],
 '1969-03-31': [array([866, 'Lúðvík Kristinsson', '1969-03-31 00:00:00.000', 'kk ', nan,
         nan, nan, nan, nan, nan, nan, nan, '2001-04-17 12:12:41.047', nan],
        dtype=object),
  array([2299, 'Lúðvík Kristinsson', '1969-03-31 00:00:00.000', 'kk ', nan,
         nan, nan, nan, nan, nan, nan, nan, '2007-04-26 11:38:05.860', nan],
        dtype=object)]}

In [11]:
reverted_back_to_dict = dict(duplicate_dict)
#reverted_back_to_dict

In [202]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [12]:
file_path = "json/einstaklingar_map.txt" ## your path variable
#duplicate_dict_json = json.dump(duplicate_dict, codecs.open(file_path, 'w', encoding='utf-8'), separators=(';', ':'), sort_keys=True, indent=4) ### this saves the array in .json format
#json_obj = json.dumps(duplicate_dict, indent = 4)
#dumped = json.dumps(duplicate_dict, cls=NumpyEncoder)
#dumped
#pd.DataFrame(reverted_back_to_dict).to_csv(file_path, encoding='utf-8-sig')
#duplicate_dict_json = json.dump(reverted_back_to_dict, codecs.open(file_path, 'w', encoding='utf-8-sig'))

#json = json.dumps(reverted_back_to_dict)
#f = open(file_path,"w")
#f.write(str(reverted_back_to_dict))
#f.close()

In [187]:
#FINAL STEP (run after everything is done):

#duplicated people put into it's own csv to be browsed later
pd.DataFrame(duplicated_einstaklingar).to_csv("csv/new/duplicated-einstaklingar.csv", encoding='utf-8-sig')
pd.DataFrame(duplicate_dict).to_csv("json/duplicate-map.json", encoding='utf-8-sig')


#save as new csv inside csv/new
pd.DataFrame(domarar).to_csv("csv/new/blak-domarar.csv", encoding='utf-8-sig')
pd.DataFrame(einstaklingar).to_csv("csv/new/blak-einstaklingar.csv", encoding='utf-8-sig')
pd.DataFrame(forsvarsmenn).to_csv("csv/new/blak-forsvarsmenn.csv.csv", encoding='utf-8-sig')
pd.DataFrame(lid).to_csv("csv/new/blak-lid.csv", encoding='utf-8-sig')
pd.DataFrame(lidimoti).to_csv("csv/new/blak-lidimoti.csv", encoding='utf-8-sig')
pd.DataFrame(lidsmenn).to_csv("csv/new/blak-lidsmenn.csv", encoding='utf-8-sig')
pd.DataFrame(lidsstjorar).to_csv("csv/new/blak-lidsstjorar.csv", encoding='utf-8-sig')
pd.DataFrame(mot).to_csv("csv/new/blak-mot.csv", encoding='utf-8-sig')
pd.DataFrame(thjalfarar).to_csv("csv/new/blak-thjalfarar.csv", encoding='utf-8-sig')